In [1]:
import os
import sys
import numpy as np
import tensorflow as tf
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from collections import Counter
import glob

print("Python:", sys.version.splitlines()[0])
print("TensorFlow:", tf.__version__)

Python: 3.12.4 (tags/v3.12.4:8e8a4ba, Jun  6 2024, 19:30:16) [MSC v.1940 64 bit (AMD64)]
TensorFlow: 2.20.0


In [4]:
import os

INPUT_ROOT = "./data"  # placeholder path

# List top entries if folder exists
if os.path.exists(INPUT_ROOT):
    print("Top entries:", os.listdir(INPUT_ROOT))
else:
    print(f"{INPUT_ROOT} does not exist. Skipping listing.")

# Dummy dataset selection logic
candidates = ["ai-project", "face-dataset"]  # placeholder candidates
DATASET_SLUG = candidates[0] if candidates else None
print("Selected dataset:", DATASET_SLUG)

# Dummy folder tree print (no actual files)
if DATASET_SLUG:
    print(f"\nFiles under {INPUT_ROOT}/{DATASET_SLUG}:")
    for i in range(3):
        print("  " * i + f"folder_level_{i}/")
        for j in range(3):
            print("  " * (i+1) + f"- file_{i}_{j}.jpg")


./data does not exist. Skipping listing.
Selected dataset: ai-project

Files under ./data/ai-project:
folder_level_0/
  - file_0_0.jpg
  - file_0_1.jpg
  - file_0_2.jpg
  folder_level_1/
    - file_1_0.jpg
    - file_1_1.jpg
    - file_1_2.jpg
    folder_level_2/
      - file_2_0.jpg
      - file_2_1.jpg
      - file_2_2.jpg


In [5]:
import os

# Step 1: Set dataset slug and subfolder (placeholders)
DATASET_SLUG = "ai-project"
SUBFOLDER = "fer data set"

dataset_root = os.path.join("./data", DATASET_SLUG)
TRAIN_DIR = os.path.join(dataset_root, SUBFOLDER, "train")
TEST_DIR  = os.path.join(dataset_root, SUBFOLDER, "test")

# Print paths (no actual files needed)
print("TRAIN_DIR:", TRAIN_DIR, "exists? (skipped check)")
print("TEST_DIR :", TEST_DIR, "exists? (skipped check)")

# Simulate folder contents
print("\nImmediate children of dataset_root:")
for i in range(3):
    print(f"  folder_level_{i}/")
    for j in range(3):
        print(f"    - file_{i}_{j}.jpg")

TRAIN_DIR: ./data\ai-project\fer data set\train exists? (skipped check)
TEST_DIR : ./data\ai-project\fer data set\test exists? (skipped check)

Immediate children of dataset_root:
  folder_level_0/
    - file_0_0.jpg
    - file_0_1.jpg
    - file_0_2.jpg
  folder_level_1/
    - file_1_0.jpg
    - file_1_1.jpg
    - file_1_2.jpg
  folder_level_2/
    - file_2_0.jpg
    - file_2_1.jpg
    - file_2_2.jpg


In [6]:
import os
import glob
from collections import Counter

# Dummy TRAIN_DIR and TEST_DIR (no real files needed)
TRAIN_DIR = "./data/ai-project/fer data set/train"
TEST_DIR  = "./data/ai-project/fer data set/test"

def collect_paths_labels(dir_path):
    paths, labels = [], []
    if not os.path.exists(dir_path):
        print(f"Warning: {dir_path} does not exist. Using dummy data.")
        # simulate 3 classes with 5 images each
        for class_name in ["happy", "sad", "angry"]:
            for i in range(5):
                paths.append(f"{dir_path}/{class_name}/img_{i}.jpg")
                labels.append(class_name)
        return paths, labels

    for class_name in sorted([d for d in os.listdir(dir_path) if os.path.isdir(os.path.join(dir_path, d))]):
        class_dir = os.path.join(dir_path, class_name)
        matched = [p for p in glob.glob(os.path.join(class_dir, "*.*")) if p.lower().endswith((".jpg", ".jpeg", ".png"))]
        if len(matched) == 0:
            print(f"Warning: class '{class_name}' has no images at {class_dir}")
            continue
        for p in matched:
            paths.append(p)
            labels.append(class_name)
    return paths, labels

train_paths, train_labels = collect_paths_labels(TRAIN_DIR)
test_paths, test_labels = collect_paths_labels(TEST_DIR)

print("Train images:", len(train_paths))
print("Test images :", len(test_paths))
print("Example train path:", train_paths[0] if train_paths else None)
print("Example train label:", train_labels[0] if train_labels else None)
print("Class distribution (train):", Counter(train_labels))

Train images: 15
Test images : 15
Example train path: ./data/ai-project/fer data set/train/happy/img_0.jpg
Example train label: happy
Class distribution (train): Counter({'happy': 5, 'sad': 5, 'angry': 5})
